<a href="https://colab.research.google.com/github/changyong93/Natural-language-processing-with-chat-bot/blob/main/%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_%EC%9E%85%EB%AC%B8(11_5_RNN%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%B6%84%EB%A5%98_%EB%82%98%EC%9D%B4%EB%B8%8C_%EB%B2%A0%EC%9D%B4%EC%A6%88_%EB%B6%84%EB%A5%98%EA%B8%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 나이브 베이즈 분류기(Naive Bayes Classifier)
---
인공신경망 알고리즘에 속하진 않지만, 머신러닝의 주요 알고리즘으롭 분류에 있어 준수한 성능을 보여줌

## 베이즈의 정리(Bayes' theorem)를 이용한 분류 메커니즘
---
입력 텍스트가 스팸인지 정상메일인지 구분하기 위한 확률
- P(정상 메일 | 입력 텍스트) = 입력 텍스트가 있을 때 정상 메일일 확률
- P(스팸 메일 | 입력 텍스트) = 입력 텍스트가 있을 때 스팸 메일일 확률  

베이지안 정리
- P(정상 메일 | 입력 텍스트) = (P(입력 텍스트 | 정상 메일) × P(정상 메일)) / P(입력 텍스트)
- P(스팸 메일 | 입력 텍스트) = (P(입력 텍스트 | 스팸 메일) × P(스팸 메일)) / P(입력 텍스트)

1) 여기서 p(입력 텍스트)는 동일 항목으로 제거
2) p(정상 메일) & p(스팸 메일)도 각각 0.5의 확률로 동일하므로 제거  

결과적으로 아래와 같음
- P(정상 메일 | 입력 텍스트) = P(입력 텍스트 | 정상 메일)
- P(스팸 메일 | 입력 텍스트) = P(입력 텍스트 | 스팸 메일)

ex)입력 텍스트의 단어가 3개인 w1,w2,w3라 가정하면
- P(정상 메일 | 입력 텍스트) = P(w1 | 정상 메일) * P(w2 | 정상 메일) * P(w3 | 정상 메일)
- P(스팸 메일 | 입력 텍스트) = P(w1 | 스팸 메일) * P(w2 | 스팸 메일) * P(w3 | 스팸 메일)  
**나이브 베이즈 분류기는 단어의 순서(sequence)가 중요하지 않음 P(정상 메일 | 입력 텍스트) = P(w1 | 정상 메일) * P(w2 | 정상 메일) * P(w3 | 정상 메일)
- P(스팸 메일 | 입력 텍스트) = P(w1 | 스팸 메일) * P(w2 | 스팸 메일) * P(w3 | 스팸 메일)  

**나이브 베이즈 분류기는 단어의 순서(sequence)가 중요하지 않음**
- 즉, BoW(bag of word, 단어의 순서는 중요하지 않고 출현 빈도(frequency)만 집중하는 텍스트 데이터의 수치화 표현)만 고려

**문제는 train data에 없는 단어는 확률이 0이므로 전체 확률이 0으로 출력됨**
- 이를 방지하기 위해 나이즈 분류기에서 분모에 스팸 메일(or정상 메일)에 나오는 모든 단어 중 중복을 제외한 전체 수를 더해주고, 분자에 +1를 더해줘서 확률이 0이 되는 것을 방지함(라플라스 스무딩)

라플라스 스무딩으로 0이 되는 것은 방지했지만, 확률값이 매우 작아져서 0으로 출력되는 문제가 발생(언더 플로우)  
**언더 플로우 방지를 위한 로그 사용**
- 양변에 모두 로그를 취하여 값을 계산
- ref : https://bcho.tistory.com/1010

## 뉴스그룹 데이터 분류하기(Classification of 20 News Group with Naive Bayes Classifier)
- 테스트 및 훈련 셋이 이미 구분되어 있음


###  뉴스그룹 데이터 이해

In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsdata = fetch_20newsgroups(subset='train')
newsdata_test = fetch_20newsgroups(subset='test')

In [ ]:
newsdata.keys()

In [ ]:
print(len(newsdata.data), len(newsdata.target), len(newsdata.target_names))

In [ ]:
print(newsdata.target_names)

In [ ]:
#첫번째 샘플 카테고리 확인
newsdata.target[0]

In [ ]:
print(newsdata.target_names[7])

In [ ]:
print(newsdata.data[0])

### 나이브 베이즈 분류
---
토큰화 후 BoW 생성

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB #다항분포 나이브 베이즈 모델
from sklearn.metrics import accuracy_score #정확도 계산

In [ ]:
dtmvector = CountVectorizer()
X_train_dtm = dtmvector.fit_transform(newsdata.data)
print(X_train_dtm.shape)

In [ ]:
tfidf_trans = TfidfTransformer() #DTM matrix를 tf-idf matrix로 변환
tfidf = tfidf_trans.fit_transform(X_train_dtm)
print(tfidf.shape)

In [ ]:
mod = MultinomialNB()
mod.fit(X = tfidf, y = newsdata.target) #alpha = 1.0 (라플라스 스무딩 적용)

In [ ]:
X_test_dtm = dtmvector.transform(newsdata_test.data) #테스트 데이터를 DTM 변환
tfidf_test = tfidf_trans.transform(X_test_dtm) #DTM을 TF-IDF로 변환

predicted = mod.predict(tfidf_test) #테스트 데이터 결과 예측
print(f"정확도(accurary : {accuracy_score(newsdata_test.target, predicted)}") #정확도 계산